In [2]:

#
# Carga del dataset
#
import pandas as pd

dataframe = pd.read_csv(
    "sentences.csv.zip",
    index_col=False,
    compression="zip",
)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#
# Exploración de los datos
#
dataframe.shape

NameError: name 'dataframe' is not defined

In [ ]:
#
# Data
#
dataframe.head()

#
# Sentimientos (clases)
#
dataframe.target.value_counts()


In [ ]:
#
# Ejemplos de frases positivas
#
for i in range(5):
    print(dataframe[dataframe.target == "positive"]["phrase"].iloc[i])
    

In [ ]:
#
# Ejemplos de frases negativas
#
for i in range(5):
    print(dataframe[dataframe.target == "negative"]["phrase"].iloc[i])

In [ ]:
#
# Ejemplos de frases neutras
#
for i in range(5):
    print(dataframe[dataframe.target == "neutral"]["phrase"].iloc[i])


In [ ]:
#
# Particionamiento de datos
#
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    dataframe.phrase,
    dataframe.target,
    test_size=0.3,
    shuffle=False,
)

In [ ]:
#
# Preprocesamiento de texto
#
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    lowercase=True,
    analyzer="word",
    token_pattern=r"\b[a-zA-Z]\w+\b",
    stop_words="english",
    max_df=0.99,
    min_df=2,
    binary=True,
)
vectorizer.fit(X_train)

In [ ]:
#
# Columnas de la matriz documento-termino
#
vectorizer.get_feature_names_out()

In [ ]:
#
# Especificación del model
#
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf


In [ ]:
#
# Entrenamiento del modelo
#

X_train_vectorized = vectorizer.transform(X_train)

clf.fit(
    X_train_vectorized,
    y_train,
)

In [ ]:
#
# Evaluación de la precisión del modelo
#
from sklearn.metrics import accuracy_score

#
# Muestra de entrenamiento
#
accuracy_score(
    y_true=y_train,
    y_pred=clf.predict(X_train_vectorized),
)


In [ ]:
#
# Muestra de prueba
#
X_test_vectorized = vectorizer.transform(X_test)
predictions = clf.predict(X_test_vectorized)

accuracy_score(
    y_true=y_test,
    y_pred=predictions,
)


In [ ]:
#
# Matriz de confusión
#
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay.from_predictions(
    y_test,
    predictions,
    cmap="Greens",
)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")


In [ ]:
#
# Almacenamiento del modelo
#
import pickle

with open("clf.pickle", "wb") as file:
    pickle.dump(clf, file)

with open("vectorizer.pickle", "wb") as file:
    pickle.dump(vectorizer, file)
    

In [ ]:
#
# Uso del modelo
#
with open("clf.pickle", "rb") as file:
    new_clf = pickle.load(file)

with open("vectorizer.pickle", "rb") as file:
    new_vectorizer = pickle.load(file)

accuracy_score(
    y_true=dataframe.target,
    y_pred=new_clf.predict(new_vectorizer.transform(dataframe.phrase)),
)
